In [1]:
!wget https://ndownloader.figshare.com/articles/11905533/versions/1 -O data.zip

--2020-04-06 18:01:14--  https://ndownloader.figshare.com/articles/11905533/versions/1
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 34.246.188.184, 18.203.214.185, 34.246.143.19, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|34.246.188.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190890195 (182M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 182.05M  5.39MB/s    in 35s     

2020-04-06 18:01:49 (5.21 MB/s) - ‘data.zip’ saved [190890195/190890195]



In [3]:
!unzip data.zip -d data

Archive:  data.zip
 extracting: data/AdultVocalizations.zip  
 extracting: data/ChickVocalizations.zip  
 extracting: data/Library_notes.pdf  


In [21]:
%%capture
!unzip data/AdultVocalizations.zip -d data/adult_vocalizations
!unzip data/ChickVocalizations.zip -d data/chick_vocalizations

In [1]:
from IPython.lib.display import Audio
import re
import pathlib
import pandas as pd
from fastcore.all import *

In [2]:
adult_paths = L(pathlib.Path('data/adult_vocalizations/').iterdir())
chick_paths = L(pathlib.Path('data/chick_vocalizations/').iterdir())

In [3]:
len(adult_paths), len(chick_paths), len(adult_paths) + len(chick_paths)

(2969, 464, 3433)

In [4]:
name_pattern = re.compile('(.*)_(.*)-(.*)-(.*)') # should extract name, data of recording, call type, rendition num

m = re.match(name_pattern, adult_paths[0].stem)
m.groups()

('WhiBlu4818', '110404', 'DC', '02')

In [5]:
Audio(filename=adult_paths[2])

In [6]:
adult_paths[0].name

'WhiBlu4818_110404-DC-02.wav'

In [7]:
paths = adult_paths + chick_paths

df = pd.DataFrame(data={'fn': [path.name for path in paths]})
df['adult'] = [path.parent.name == 'adult_vocalizations' for path in paths]

In [8]:
df.head()

,fn,adult
0,WhiBlu4818_110404-DC-02.wav,True
1,BluRas61dd_110502-AggC-15.wav,True
2,LblRed0613_110920-So-37.wav,True
3,LblBla4548_130416-So-02.wav,True
4,LblRed0613_110920-NestC-48.wav,True


Unfortunately, there is an issue with naming but it can easily be corrected.There are quite a few names that have an extra underscore: GreRas2400_110615_TetC-28.wav instead of GreRas2400_110615-TetC-28.wav.

We can work around this by modifying our regex pattern.

In [9]:
name_pattern = re.compile('(.*)_(.*)[-_](.*)-(.*)\.wav') # should extract name, data of recording, call type, rendition num

In [10]:
name_date_type_num = L(re.match(name_pattern, path.name).groups() for path in paths)

In [11]:
name, date, call_type, rendition_num = [list(l) for l in zip(*name_date_type_num)]

In [12]:
name[:5], date[:5], call_type[:5], rendition_num[:5]

(['WhiBlu4818', 'BluRas61dd', 'LblRed0613', 'LblBla4548', 'LblRed0613'],
 ['110404', '110502', '110920', '130416', '110920'],
 ['DC', 'AggC', 'So', 'So', 'NestC'],
 ['02', '15', '37', '02', '48'])

Some manual clean up of infrequently occuring issues

In [13]:
date[923], date[2798]

('110608-Nest', '110608-Nest')

In [14]:
date[923] = '110608'; date[2798] = '110608'

In [15]:
call_type[923] = 'Ne'; call_type[2798] = 'Ne'

In [16]:
date[926], date[1209], date[1884] = [None ] * 3

In [17]:
df['name'] = name
df['date_recorded'] = [pd.to_datetime(d, yearfirst=True, errors='coerce') for d in date]
df['call_type'] = call_type
df['rendition_num'] = rendition_num

It also seems that there were some recordings classified as coming both from chicks and adults

In [18]:
df.fn.nunique()

3405

That's just 14 recordings that have been assigned both labels. Let's remove them from the dataset.

In [19]:
df = df[~df.fn.duplicated()]

In [20]:
df.reset_index(inplace=True, drop=True)

In [21]:
sorted(df.call_type.unique())

['Ag',
 'AggC',
 'BeggSeq',
 'Beggseq',
 'DC',
 'DisC',
 'LTC',
 'Ne',
 'NeArkC',
 'NeKakleC',
 'NeSeq',
 'NekakleC',
 'NestC',
 'NestCSeq',
 'NestCseq',
 'NestSeq',
 'So',
 'Song',
 'Te',
 'Tet',
 'TetC',
 'ThuC',
 'ThuckC',
 'ThukC',
 'TukC',
 'WC',
 'Wh',
 'Whi',
 'WhiC',
 'WhiCNestC',
 'Whine',
 'WhineC',
 'WhineCSeq']

As mentioned in the library notes ('data/Library_notes.pdf') that came with the data, there are issues with the labels.

Some of the malformed labels are ambiguous. For instance, is 'ThuC' a 'Thuk' or 'Tuck'? What about 'Thuck? Is 'DisC' a 'Distance' call or a 'Distress' call?

We have no way of telling unless we listen to the calls ourselves and make educated guesses. Let's do a preliminary, high level clean up as we work on unifying the labels. Later, once we have a trained model we will be able to leverage it to further identify any potential issues.

But how do we go about the initial data clean up? We could load recordings one by one, but that gets tedious quickly and it is also easy to make a mistake. To be efficient and effective at this task, we could use a tool that could give us instantenous access to calls of a particular type, so that we can easily compare them.

Let's hack something like this together using ipywidgets.

Let's move all the files into a single directory so that we have an easier time working with the data.

In [22]:
!mkdir data/vocalizations

!cp data/adult_vocalizations/* data/vocalizations/
!cp data/chick_vocalizations/* data/vocalizations/

mkdir: cannot create directory ‘data/vocalizations’: File exists


In [23]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [24]:
def create_expanded_button(description, button_style):
    return widgets.Button(
        description=description,
        button_style=button_style,
        layout=widgets.Layout(height='auto', width='auto')
    )

In [25]:
grid = widgets.GridspecLayout(5, 8)
call_types = sorted(df.call_type.unique())

out = widgets.Output(layout={'border': '1px solid black', 'margin': '20px 0px 0px 0px'})

@out.capture(clear_output=True)
def update_output(button):
    lstrip = button.description.find('[')
    call_type = button.description[:lstrip-1]
    display(widgets.HTML(f'<p>Loaded call of type <strong>{call_type}</strong></p>'))
    fn = df[df.call_type == call_type].sample(1).fn.item()
    path = f'data/vocalizations/{fn}'

    with open(path, 'rb') as fd:
        contents = fd.read()
        audio = widgets.Audio(value=contents, autoplay=True, loop=False, controls=True)
    
    display(audio)

for i in range(5):
    for j in range(8):
        idx = i*8 + j
        if idx < len(call_types):
            n = df[df.call_type == call_types[idx]].shape[0]
            btn = create_expanded_button(f'{call_types[idx]} [{n}]', button_style='warning')
            btn.on_click(update_output)
            grid[i, j] = btn
    
initial_button = random.sample(grid.children, 1)[0] # initialize to some random value
update_output(initial_button)
    
widgets.VBox([widgets.HTML('<h3>Press to load a random recording with a given label</h3>'), grid, out])

In [27]:
type2name = {
    'Ag': 'Wsst',
    'Be': 'Begging',
    'DC': 'Distance',
    'Di': 'Distress',
    'LT': 'Long Tonal',
    'Ne': 'Nest',
    'So': 'Song',
    'Te': 'Tet',
    'Th': 'Thuk',
    'Tu': 'Tuck',
    'Wh': 'Whine'
}

In [28]:
df.to_csv('data/annotations.csv', index=False)